<a href="https://colab.research.google.com/github/StarLord598/Interesting_Notebook_Exercises/blob/main/Andres_Version_Agentic_RAG_Hands_On.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This hands on experiment will enable you to implement Search engine->LLM for accurate QA.
https://medium.com/the-ai-forum/implementing-agentic-rag-using-langchain-b22af7f6a3b5

In [1]:
!pip install -qU langchain langchain_openai langgraph arxiv duckduckgo-search
!pip install -qU faiss-cpu pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta

In [2]:
!pip install --upgrade tiktoken

In [3]:
!pip install langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.5 MB/s eta 0:00:00


In [5]:
import os

# os.environ['OPENAI_API_KEY'] =  "<YOUR KEY HERE>"

In [6]:
# Step 1: Setup the environment and variables
from google.colab import userdata
from uuid import uuid4
import os
#
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE1 - LangGraph - {uuid4().hex[0:8]}"

# os.environ["LANGCHAIN_API_KEY"] =  "<YOUR KEY HERE>"

In [7]:
#Step 2: Simple Retrieval chain. This will error out if you have $0 credit for yout openai key
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# Load the document pertaining to a particular topic
docs = ArxivLoader(query="Retrieval Augmented Generation", load_max_docs=2).load()

# Split the dpocument into smaller chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=10
)

chunked_documents = text_splitter.split_documents(docs)
#
# Instantiate the Embedding Model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",openai_api_key=os.environ['OPENAI_API_KEY'])
# Create Index- Load document chunks into the vectorstore
faiss_vectorstore = FAISS.from_documents(
    documents=chunked_documents,
    embedding=embeddings,
)
# Create a retriver
retriever = faiss_vectorstore.as_retriever()

In [8]:
#Step 3: generate RAG
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Use the following context to answer the user's query. If you cannot answer the question, please respond with 'I don't know'.

Question:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [9]:
#Step 4: Start the LLM
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")

In [10]:
#Step 5: Build a RAG chain
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_generation_chain = (
       {"context": itemgetter("question")
    | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | openai_chat_model, "context": itemgetter("context")}
)
#
retrieval_augmented_generation_chain

{
  context: RunnableLambda(itemgetter('question'))
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7d0ddd3ac370>),
  question: RunnableLambda(itemgetter('question'))
}
| RunnableAssign(mapper={
    context: RunnableLambda(itemgetter('context'))
  })
| {
    response: ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following context to answer the user's query. If you cannot answer the question, please respond with 'I don't know'.\n\nQuestion:\n{question}\n\nContext:\n{context}\n"))])
              | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7d0ddd2a3df0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7d0ddd2a1a20>, openai_api_key=SecretStr('**********'), openai_proxy=''),
    context: RunnableLambda(ite

In [11]:
#Step 6: Ask a Question
await retrieval_augmented_generation_chain.ainvoke({"question" : "Who wrote the paper on retrieval augmentation generation?"})

{'response': AIMessage(content='Huayang Li, Yixuan Su, Deng Cai, Yan Wang, and Lemao Liu wrote the paper on retrieval augmentation generation.', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 1226, 'total_tokens': 1255}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1b0a1774-1dc4-4c70-8130-9c4f4ac18333-0', usage_metadata={'input_tokens': 1226, 'output_tokens': 29, 'total_tokens': 1255}),
 'context': [Document(page_content='Retrieval-Augmented Paradigm\nIn this section, we ﬁrst give a general formulation\nof retrieval-augmented text generation. Then, we\ndiscuss three major components of the retrieval-\naugmented generation paradigm, including the re-\narXiv:2202.01110v2  [cs.CL]  13 Feb 2022\nInput\nSources \n(Sec. 2.2):\nTraining \nCorpus', metadata={'Published': '2022-02-13', 'Title': 'A Survey on Retrieval-Augmented Text Generation', 'Authors': 'Huayang Li, Yixuan Su, Deng Cai, Yan Wang, L

In [12]:
await retrieval_augmented_generation_chain.ainvoke({"question" : "What is Extreme RAG in the paper ERATTA in arxiv?"})

{'response': AIMessage(content="I don't know.", response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 1353, 'total_tokens': 1358}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c5d380a2-0ee1-4196-bf47-37e397ff0f79-0', usage_metadata={'input_tokens': 1353, 'output_tokens': 5, 'total_tokens': 1358}),
 'context': [Document(page_content='DuetRAG is not only to equip models with domain-specific knowledge but also to enable them to utilize external\ndocuments to obtain answers when internal knowledge is uncertain. By complementing internal and external knowledge,\nDuetRAG aims to enhance the robustness of the model. We provide a detailed description of the DuetRAG methodology\narXiv:2405.13002v1  [cs.CL]  12 May 2024', metadata={'Published': '2024-05-12', 'Title': 'DuetRAG: Collaborative Retrieval-Augmented Generation', 'Authors': 'Dian Jiao, Li Cai, Jingsheng Huang, Wenqiao Zhang, Siliang Tang, Yueting Zh

# So we see our LLM has limited knowledge that needs to be enhanced

In [13]:
#Step 7: Creating a Tool belt with DuckduckGo web serach and arxiv
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langgraph.prebuilt import ToolExecutor
tool_belt = [
    DuckDuckGoSearchRun(),
    ArxivQueryRun()
]

tool_executor = ToolExecutor(tool_belt)

In [14]:
#Step 8: Instantiate openai function calling
from langchain_core.utils.function_calling import convert_to_openai_function
#
model = ChatOpenAI(temperature=0)
#
functions = [convert_to_openai_function(t) for t in tool_belt]
model = model.bind_functions(functions)

In [15]:
#Step 9: Instantiate Langgraph to pass information between various stages of the nodes
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]

In [16]:
#Step 10: Build Nodes, call_model and call_tool
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = tool_executor.invoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

In [17]:
#Step 11: Built workflow and set agent
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)
workflow.set_entry_point("agent")


In [18]:
#Step 12: Build a conditional edge for routing
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)



In [19]:
#connect conditional edge to agent
workflow.add_edge("action", "agent")

In [20]:
#Step 13: Compile the graph
app = workflow.compile()
app

CompiledStateGraph(nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden']}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<messages>(recurse=True, writes=[ChannelWriteEntry(channel='messages', value=<object object at 0x7d0de2af8330>, skip_none=False, mapper=_get_state_key(recurse=False))], require_at_least_one_of=['messages']), ChannelWrite<start:agent>(recurse=True, writes=[ChannelWriteEntry(channel='start:agent', value='__start__', skip_none=False, mapper=None)], require_at_least_one_of=None)]), 'agent': PregelNode(config={'tags': []}, channels={'messages': 'messages'}, triggers=['start:agent', 'action'], mapper=functools.partial(<function _coerce_state at 0x7d0ddd0c5ab0>, <class '__main__.AgentState'>), writers=[ChannelWrite<agent,messages>(recurse=True, writes=[ChannelWriteEntry(channel='agent', value='agent', skip_none=False, mapper=None), ChannelWriteEntry(channel='messages', value=<object object at 0x7d0de2af8330>, skip_none=False, mapper=_get

In [21]:
# Step 14: Invoke Ask a Query
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who wrote the paper ERATTA in arxiv?")]}

response = app.invoke(inputs)
print(response['messages'][-1].content)

The paper "ERATTA: Extreme RAG for Table To Answers with Large Language Models" on arXiv was written by Sohini Roychowdhury, Marko Krema, Anvar Mahammad, Brian Moore, Arijit Mukherjee, and Punit Prakashchandra.


In [22]:
#Vary teh question
question = "What is Extreme RAG in the paper ERATTA in arxiv?"

inputs = {"messages" : [HumanMessage(content=question)]}
#
response = app.invoke(inputs)
print(response['messages'][-1].content)


The paper "ERATTA: Extreme RAG for Table To Answers with Large Language Models" proposes a unique Large Language Model (LLM)-based system where multiple LLMs can be invoked to enable data authentication, user query routing, data retrieval, and custom prompting for question-answering capabilities from data tables that are highly varying and large in size. The system is tuned to extract information from Enterprise-level data products and provide real-time responses under 10 seconds. It includes a five metric scoring module that detects and reports hallucinations in the LLM responses. The proposed system and scoring metrics achieve >90% confidence scores across hundreds of user queries in sustainability, financial health, and social media domains. Extensions to the proposed extreme RAG architectures can enable heterogeneous source querying using LLMs.
